In [26]:
# Install necessary packages
!pip install transformers sentencepiece torch torchvision albumentations timm


In [27]:
# Import required libraries
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import albumentations as A
import numpy as np
import pandas as pd
import itertools
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import cv2
import timm

In [28]:

# Step 1: Mount Google Drive to access the dataset.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# Define paths to the image folder and caption file
image_folder = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/Flicker8k_Dataset'
caption_file = '/content/drive/MyDrive/Bangla Image dataset with caption/Flickr8k_Dataset/BAN-Cap_captiondata.json'


In [30]:
# Define configuration with adjusted batch size
class CFG:
    model_name = "resnet50"
    text_encoder_model = "csebuetnlp/banglabert"  # Bangla BERT model
    pretrained = True
    trainable = True
    batch_size = 16  # Reduced batch size
    size = 224
    image_embedding = 2048
    text_embedding = 768
    projection_dim = 512
    max_length = 128
    temperature = 0.07
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    head_lr = 1e-3
    weight_decay = 1e-4
    patience = 2
    factor = 0.5
    device = "cuda" if torch.cuda.is_available() else "cpu"
    epochs = 50


In [31]:
# Dataset class
class CLIPDataset(Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        self.image_filenames = image_filenames
        self.captions = list(captions)
        self.encoded_captions = tokenizer(
            list(captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }
        image = cv2.imread(f"{image_folder}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['caption'] = self.captions[idx]
        return item

    def __len__(self):
        return len(self.captions)

In [32]:

# Image Encoder
class ImageEncoder(nn.Module):
    def __init__(self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained, num_classes=0, global_pool="avg")
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [33]:
# Text Encoder
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        self.model = AutoModel.from_pretrained(model_name)
        for p in self.model.parameters():
            p.requires_grad = trainable
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

In [34]:
# Projection Head
class ProjectionHead(nn.Module):
    def __init__(self, embedding_dim, projection_dim=CFG.projection_dim, dropout=0.1):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [35]:

# CLIP Model
class CLIPModel(nn.Module):
    def __init__(self, temperature=CFG.temperature):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=CFG.image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=CFG.text_embedding)
        self.temperature = temperature

    def forward(self, batch):
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax((images_similarity + texts_similarity) / 2 * self.temperature, dim=-1)
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss = (images_loss + texts_loss) / 2.0
        return loss.mean()

In [36]:

# Cross Entropy
def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    return loss.mean() if reduction == "mean" else loss

# Data loaders
def get_transforms():
    return A.Compose([
        A.Resize(CFG.size, CFG.size, always_apply=True),
        A.Normalize(max_pixel_value=255.0, always_apply=True),
    ])

In [37]:
with open(caption_file, 'r', encoding='utf-8') as f:
    captions_data = json.load(f)

# Print a sample item to inspect the structure
print(captions_data[0])  # Check the first item for key names


{'caption_id': '1000268201_693b08cb0e.jpg#0', 'english_caption': 'A child in a pink dress is climbing up a set of stairs in an entry way .', 'bengali_caption': 'একটি গোলাপী জামা পরা বাচ্চা মেয়ে একটি বাড়ির প্রবেশ পথের সিঁড়ি বেয়ে উঠছে।'}


In [38]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  # Add this line for functional operations
import albumentations as A
import numpy as np
import pandas as pd
import itertools
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import cv2
import timm


In [42]:
import torch
from tqdm import tqdm
import itertools
import json
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import torch.nn.functional as F

# Optimized train_epoch with mixed precision
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    model.train()
    scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler
    loss_meter = 0
    correct_predictions = 0
    total_predictions = 0
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}

        with torch.cuda.amp.autocast():  # Mixed precision training
            loss, outputs = model(batch)  # Ensure model outputs a tensor, not a scalar

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if step == "batch":
            lr_scheduler.step()

        loss_meter += loss.item()

        # Ensure that outputs are a tensor of shape [batch_size, num_classes] for classification tasks
        if outputs.dim() == 1:  # If outputs is a vector of shape [batch_size]
            outputs = outputs.unsqueeze(1)  # Add extra dimension if necessary

        # Accuracy calculation (assuming binary classification or multi-class classification)
        predictions = torch.argmax(outputs, dim=-1)  # Assuming model outputs logits or probabilities
        labels = batch["labels"]  # Assuming 'labels' are in the batch

        # Ensure labels have the correct shape as well
        if labels.dim() == 1:  # If labels are in a flat vector form, ensure they're compatible
            labels = labels.unsqueeze(1)

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

    accuracy = correct_predictions / total_predictions
    return loss_meter / len(train_loader), accuracy

# Validation function with accuracy calculation
def valid_epoch(model, valid_loader):
    model.eval()  # Set the model to evaluation mode
    loss_meter = 0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():  # Disable gradient computation
        for batch in tqdm(valid_loader, total=len(valid_loader)):
            batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}

            # No need for mixed precision during validation
            loss, outputs = model(batch)  # Ensure model outputs a tensor, not a scalar

            loss_meter += loss.item()

            # Ensure that outputs are a tensor of shape [batch_size, num_classes] for classification tasks
            if outputs.dim() == 1:  # If outputs is a vector of shape [batch_size]
                outputs = outputs.unsqueeze(1)  # Add extra dimension if necessary

            # Accuracy calculation (assuming binary classification or multi-class classification)
            predictions = torch.argmax(outputs, dim=-1)  # Assuming model outputs logits or probabilities
            labels = batch["labels"]

            # Ensure labels have the correct shape as well
            if labels.dim() == 1:  # If labels are in a flat vector form, ensure they're compatible
                labels = labels.unsqueeze(1)

            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    accuracy = correct_predictions / total_predictions
    return loss_meter / len(valid_loader), accuracy

# Main function with optimizations for data loading, mixed precision training, early stopping, and accuracy
def main():
    with open(caption_file, 'r', encoding='utf-8') as f:
        captions_data = json.load(f)

    # Extract image files and captions based on provided structure
    image_files = [item['caption_id'].split('#')[0] for item in captions_data]
    captions = [item['bengali_caption'] for item in captions_data]

    tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)
    transforms = get_transforms()
    dataset = CLIPDataset(image_files, captions, tokenizer, transforms)

    # Data splitting into train and validation sets
    train_size = int(0.8 * len(dataset))
    valid_size = len(dataset) - train_size
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

    # DataLoader with num_workers for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)

    model = CLIPModel().to(CFG.device)

    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(model.image_projection.parameters(), model.text_projection.parameters()),
         "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
    ]

    optimizer = torch.optim.AdamW(params, weight_decay=0.0)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=CFG.patience, factor=CFG.factor)
    step = "epoch"

    best_loss = float('inf')
    best_epoch = 0
    no_improvement_epochs = 0  # Track how many epochs without improvement
    patience = 5  # Stop after 5 epochs of no improvement

    # For tracking loss and accuracy for plotting later
    train_losses, valid_losses = [], []
    train_accuracies, valid_accuracies = [], []

    for epoch in range(CFG.epochs):
        print(f"Epoch {epoch + 1}/{CFG.epochs}")

        # Train the model
        train_loss, train_accuracy = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy * 100:.2f}%")

        # Validate the model
        valid_loss, valid_accuracy = valid_epoch(model, valid_loader)
        print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy * 100:.2f}%")

        # Append to tracking lists for plotting later
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        train_accuracies.append(train_accuracy)
        valid_accuracies.append(valid_accuracy)

        # Early stopping check: if both losses are increasing, stop training
        if valid_loss > best_loss and train_loss > best_loss:
            no_improvement_epochs += 1
        else:
            no_improvement_epochs = 0

        # Save the best model based on validation loss
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch
            torch.save(model.state_dict(), "best_clip_model_bangla.pt")
            print("Best model saved!")

        # Early stopping: if no improvement for `patience` epochs, stop training
        if no_improvement_epochs >= patience:
            print(f"Early stopping triggered. No improvement for {patience} epochs.")
            break

        if step == "epoch":
            lr_scheduler.step(valid_loss)

    print(f"Training finished. Best model found at epoch {best_epoch + 1} with validation loss {best_loss:.4f}.")

    # Plotting Training and Validation Loss and Accuracy
    import matplotlib.pyplot as plt

    # Plot Train and Validation Loss
    plt.figure(figsize=(12, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(valid_losses, label='Valid Loss')
    plt.title("Loss During Training")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    # Plot Train and Validation Accuracy
    plt.figure(figsize=(12, 6))
    plt.plot(train_accuracies, label='Train Accuracy')
    plt.plot(valid_accuracies, label='Valid Accuracy')
    plt.title("Accuracy During Training")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

# Run the training with optimizations, early stopping, and accuracy tracking
main()


<ipython-input-42-937f8896dcb0>:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler


Epoch 1/50


  0%|          | 0/2023 [00:00<?, ?it/s]<ipython-input-42-937f8896dcb0>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed precision training
  0%|          | 0/2023 [00:00<?, ?it/s]


TypeError: iteration over a 0-d tensor

In [41]:
import torch
from tqdm import tqdm
import itertools
import json
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
import torch.nn.functional as F

# Optimized train_epoch with mixed precision
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    model.train()
    scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler
    loss_meter = 0
    correct_predictions = 0
    total_predictions = 0
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}

        with torch.cuda.amp.autocast():  # Mixed precision training
            loss, outputs = model(batch)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if step == "batch":
            lr_scheduler.step()

        loss_meter += loss.item()

        # Accuracy calculation (assuming binary classification or multi-class classification)
        predictions = torch.argmax(outputs, dim=-1)  # Assuming model outputs logits or probabilities
        labels = batch["labels"]  # Assuming 'labels' are in the batch
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

    accuracy = correct_predictions / total_predictions
    return loss_meter / len(train_loader), accuracy

# Validation function with accuracy calculation
def valid_epoch(model, valid_loader):
    model.eval()  # Set the model to evaluation mode
    loss_meter = 0
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():  # Disable gradient computation
        for batch in tqdm(valid_loader, total=len(valid_loader)):
            batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}

            # No need for mixed precision during validation
            loss, outputs = model(batch)

            loss_meter += loss.item()

            # Accuracy calculation (assuming binary classification or multi-class classification)
            predictions = torch.argmax(outputs, dim=-1)  # Assuming model outputs logits or probabilities
            labels = batch["labels"]
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    accuracy = correct_predictions / total_predictions
    return loss_meter / len(valid_loader), accuracy

# Main function with optimizations for data loading, mixed precision training, early stopping, and accuracy
def main():
    with open(caption_file, 'r', encoding='utf-8') as f:
        captions_data = json.load(f)

    # Extract image files and captions based on provided structure
    image_files = [item['caption_id'].split('#')[0] for item in captions_data]
    captions = [item['bengali_caption'] for item in captions_data]

    tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)
    transforms = get_transforms()
    dataset = CLIPDataset(image_files, captions, tokenizer, transforms)

    # Data splitting into train and validation sets
    train_size = int(0.8 * len(dataset))
    valid_size = len(dataset) - train_size
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

    # DataLoader with num_workers for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)

    model = CLIPModel().to(CFG.device)

    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(model.image_projection.parameters(), model.text_projection.parameters()),
         "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
    ]

    optimizer = torch.optim.AdamW(params, weight_decay=0.0)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=CFG.patience, factor=CFG.factor)
    step = "epoch"

    best_loss = float('inf')
    best_epoch = 0
    no_improvement_epochs = 0  # Track how many epochs without improvement
    patience = 5  # Stop after 5 epochs of no improvement

    # For tracking loss and accuracy for plotting
    train_losses, valid_losses = [], []
    train_accuracies, valid_accuracies = [], []

    for epoch in range(CFG.epochs):
        print(f"Epoch {epoch + 1}/{CFG.epochs}")

        # Train the model
        train_loss, train_accuracy = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy * 100:.2f}%")

        # Validate the model
        valid_loss, valid_accuracy = valid_epoch(model, valid_loader)
        print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy * 100:.2f}%")

        # Append to tracking lists for plotting later
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        train_accuracies.append(train_accuracy)
        valid_accuracies.append(valid_accuracy)

        # Early stopping check: if both losses are increasing, stop training
        if valid_loss > best_loss and train_loss > best_loss:
            no_improvement_epochs += 1
        else:
            no_improvement_epochs = 0

        # Save the best model based on validation loss
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch
            torch.save(model.state_dict(), "best_clip_model_bangla.pt")
            print("Best model saved!")

        # Early stopping: if no improvement for `patience` epochs, stop training
        if no_improvement_epochs >= patience:
            print(f"Early stopping triggered. No improvement for {patience} epochs.")
            break

        if step == "epoch":
            lr_scheduler.step(valid_loss)

    print(f"Training finished. Best model found at epoch {best_epoch + 1} with validation loss {best_loss:.4f}.")

    # Plotting Training and Validation Loss and Accuracy
    import matplotlib.pyplot as plt

    # Plot Train and Validation Loss
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
    plt.plot(range(1, len(valid_losses) + 1), valid_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    # Plot Train and Validation Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, len(valid_accuracies) + 1), valid_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Training and Validation Accuracy')

    plt.tight_layout()
    plt.show()

# Run the training with optimizations, early stopping, and accuracy tracking
main()


<ipython-input-41-9fe4e134bd14>:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler


Epoch 1/50


  0%|          | 0/2023 [00:00<?, ?it/s]<ipython-input-41-9fe4e134bd14>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed precision training
  0%|          | 0/2023 [00:00<?, ?it/s]


TypeError: iteration over a 0-d tensor

In [39]:
import torch
from tqdm import tqdm

# Validation function for the model
def valid_epoch(model, valid_loader):
    model.eval()  # Set the model to evaluation mode
    loss_meter = 0
    with torch.no_grad():  # Disable gradient computation
        for batch in tqdm(valid_loader, total=len(valid_loader)):
            batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}

            # No need for mixed precision during validation
            loss = model(batch)

            loss_meter += loss.item()

    # Return the average loss
    return loss_meter / len(valid_loader)


In [40]:
import torch
from tqdm import tqdm



# Optimized train_epoch with mixed precision
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    model.train()
    scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler
    loss_meter = 0
    for batch in tqdm(train_loader, total=len(train_loader)):
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}

        with torch.cuda.amp.autocast():  # Mixed precision training
            loss = model(batch)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if step == "batch":
            lr_scheduler.step()

        loss_meter += loss.item()
    return loss_meter / len(train_loader)

# Validation function remains the same

# Main function with optimizations for data loading and mixed precision training
def main():
    with open(caption_file, 'r', encoding='utf-8') as f:
        captions_data = json.load(f)

    # Extract image files and captions based on provided structure
    image_files = [item['caption_id'].split('#')[0] for item in captions_data]
    captions = [item['bengali_caption'] for item in captions_data]

    tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)
    transforms = get_transforms()
    dataset = CLIPDataset(image_files, captions, tokenizer, transforms)

    # Data splitting into train and validation sets
    train_size = int(0.8 * len(dataset))
    valid_size = len(dataset) - train_size
    train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

    # DataLoader with num_workers for faster data loading
    train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)

    model = CLIPModel().to(CFG.device)

    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(model.image_projection.parameters(), model.text_projection.parameters()),
         "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
    ]

    optimizer = torch.optim.AdamW(params, weight_decay=0.0)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=CFG.patience, factor=CFG.factor)
    step = "epoch"

    best_loss = float('inf')
    for epoch in range(CFG.epochs):
        print(f"Epoch {epoch + 1}/{CFG.epochs}")

        train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
        print(f"Train Loss: {train_loss:.4f}")

        valid_loss = valid_epoch(model, valid_loader)
        print(f"Validation Loss: {valid_loss:.4f}")

        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), "best_clip_model_bangla.pt")
            print("Best model saved!")

        if step == "epoch":
            lr_scheduler.step(valid_loss)

# Run the training with optimizations
main()

<ipython-input-40-1f09fcd16a43>:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Mixed precision scaler


Epoch 1/50


  0%|          | 0/2023 [00:00<?, ?it/s]<ipython-input-40-1f09fcd16a43>:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # Mixed precision training
100%|██████████| 2023/2023 [03:49<00:00,  8.83it/s]


Train Loss: 23.9090


100%|██████████| 506/506 [00:34<00:00, 14.86it/s]


...

Train Loss: 2.4677


100%|██████████| 506/506 [00:34<00:00, 14.83it/s]


Validation Loss: 2.7767
Best model saved!
Epoch 6/50


  4%|▍         | 77/2023 [00:10<04:25,  7.34it/s]


KeyboardInterrupt: 

#iNTERFACE

In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm

# Assuming CLIPDataset and CFG have already been defined
# Ensure the paths to the dataset are properly set

# Load and prepare the dataset
with open(caption_file, 'r', encoding='utf-8') as f:
    captions_data = json.load(f)

# Prepare image files and captions
image_files = [item['caption_id'].split('#')[0] for item in captions_data]
captions = [item['bengali_caption'] for item in captions_data]

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)
transforms = get_transforms()

# Create the dataset
dataset = CLIPDataset(image_files, captions, tokenizer, transforms)

# Split dataset into train and validation sets
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
_, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

# Define the function to get image embeddings
def get_image_embeddings(valid_dataset, model_path):
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=4)

    # Load the trained model
    model = CLIPModel().to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))
    model.eval()

    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model.image_encoder(batch["image"].to(CFG.device))
            image_embeddings = model.image_projection(image_features)
            valid_image_embeddings.append(image_embeddings)

    return model, torch.cat(valid_image_embeddings)

# Perform inference to get image embeddings from the validation set
model, image_embeddings = get_image_embeddings(valid_dataset, "/content/best_clip_model_bangla.pt")


In [ ]:
import matplotlib.pyplot as plt  # Add this import


In [ ]:
def find_matches(model, image_embeddings, query, image_files, n=9):
    tokenizer = AutoTokenizer.from_pretrained(CFG.text_encoder_model)

    # Encode the query and print to ensure uniqueness
    encoded_query = tokenizer([query], return_tensors="pt", padding=True, truncation=True, max_length=CFG.max_length)
    print(f"Encoded Query: {encoded_query}")  # Check if different prompts yield different encodings

    batch = {
        key: torch.tensor(values).to(CFG.device)
        for key, values in encoded_query.items()
    }

    with torch.no_grad():
        text_features = model.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        text_embeddings = model.text_projection(text_features)

    # Normalize embeddings and print for debugging
    text_embeddings_n = F.normalize(text_embeddings, p=2, dim=-1)
    image_embeddings_n = F.normalize(image_embeddings, p=2, dim=-1)

    print(f"Text Embeddings: {text_embeddings_n}")  # Check if text embeddings change with different prompts

    # Calculate similarity and retrieve top matches
    dot_similarity = text_embeddings_n @ image_embeddings_n.T
    values, indices = torch.topk(dot_similarity.squeeze(0), n)

    matches = [image_files[idx] for idx in indices]
    print(f"Top match values: {values}")  # To see if similarity scores vary

    # Display the matched images
    _, axes = plt.subplots(3, 3, figsize=(10, 10))
    for match, ax in zip(matches, axes.flatten()):
        image = cv2.imread(f"{image_folder}/{match}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        ax.axis("off")

    plt.show()

In [ ]:
prompt = "কুকুর "  # Example prompt for "A dog is playing"
find_matches(model, image_embeddings, prompt, image_files)


In [ ]:
from google.colab import files

# Path to your saved model file
model_path = "best_clip_model_bangla.pt"  # Replace with your actual model file name if different

# Download the file
files.download(model_path)


In [ ]:
# Import the necessary libraries
import shutil
from google.colab import files

# Path to the model file and the output zip file
model_path = "best_clip_model_bangla.pt"  # Replace with your actual model file name
zip_model_path = "best_clip_model_bangla.zip"  # Name of the output zip file

# Zip the model file
shutil.make_archive(zip_model_path.replace('.zip', ''), 'zip', '.', model_path)

# Download the zipped file
files.download(zip_model_path)


In [ ]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import json
import itertools
from torch.utils.data import DataLoader
from transformers import AutoTokenizer


In [ ]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import json
import itertools
from torch.utils.data import DataLoader
from transformers import AutoTokenizer



In [ ]:
import matplotlib.pyplot as plt

# Function to plot the training and validation losses
def plot_loss(train_losses, valid_losses):
    """
    Plots the training and validation losses.

    Arguments:
    - train_losses: List of training losses.
    - valid_losses: List of validation losses.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss", color='blue', marker='o')
    plt.plot(valid_losses, label="Validation Loss", color='red', marker='x')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Losses')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# This function will be called after the `main()` function to plot the losses
# Collect losses in the `main()` loop to pass to this function.

# Example usage after training:
# plot_loss(train_losses, valid_losses)
